In [24]:
'''IMPORTING NECESSARY FUNCTIONS FROM GENERAL FUNCTION FILE'''

%run GeneralFunctions.ipynb

# Calculation Functions Figure 6

In [21]:
def maleFemaleComaprLst(comparLst):
    maleComparLst = []
    femaleComparLst = []
    
    for lst in comparLst:
        condition = lst[0]
        sample1_name = lst[1]
        sample2_name = lst[2]
        
        sample1_sex = mouseSex(sample1_name)
        sample2_sex = mouseSex(sample2_name)
        
        if sample1_sex == sample2_sex:
            if sample1_sex == 'male': maleComparLst.append(lst)
            else: femaleComparLst.append(lst)
    
    return(maleComparLst, femaleComparLst)

In [23]:
def concatMandFPlasticCompar(comparLst):
    maleComparLst, femaleComparLst = maleFemaleComaprLst(plastComparLst)
    
    plastComparFvsM = []
    
    # Iterating through female samples 
    for femaleLst in femaleComparLst:
        female_condition = femaleLst[0]
        female_sample1_name = femaleLst[1]
        female_sample2_name = femaleLst[2]
        
        # Finding the sample condition in male sampels
        for maleLst in maleComparLst:
            if female_condition == maleLst[0]:
                male_condition = maleLst[0]
                male_sample1_name = maleLst[1]
                male_sample2_name = maleLst[2]
        
        plastComparFvsM.append([female_condition, female_sample1_name, female_sample2_name, male_sample1_name, male_sample2_name])
        
        print(f'{male_condition}_{male_sample1_name}_{male_sample2_name}')
        print(f'{female_condition}_{female_sample1_name}_{female_sample2_name}')
        
        # Reading in AnnData Objects 
        maleObj = sc.read(f'F:/SampleData/PlasticityComparisons/{female_condition}_{male_sample1_name}_{male_sample2_name}.h5ad')
        femaleObj = sc.read(f'F:/SampleData/PlasticityComparisons/{female_condition}_{female_sample1_name}_{female_sample2_name}.h5ad')
        
        # Concatenating data 
        concatData = sc.concat([maleObj, femaleObj], axis=0)
        
        # Adds missing field to 'data.uns' that is necessary for running the ranked_sum_genes function
        concatData.uns['log1p'] = {'base': None}
        
        # Re-calculating gene info after concatenating the samples
        sc.pp.calculate_qc_metrics(concatData, percent_top=None, log1p=False, inplace=True)

        # Calculating DE Scores 
        sc.tl.rank_genes_groups(concatData, 'sex', method='wilcoxon')
        
        # Re-Clustering
        sc.tl.pca(concatData, svd_solver='arpack')
        sc.pp.neighbors(concatData, n_neighbors=50, n_pcs =50, use_rep = "Scanorama")
        sc.tl.umap(concatData)
        sc.tl.leiden(concatData, resolution = 0.3, key_added = "leiden_0.3")
        
        # Saving concat AnnData file 
        concatData.write(f'F:/SampleData/PlasitcityComparisons_FvsM/{female_condition}.h5ad')
    
    return(plastComparFvsM)

In [ ]:
def addingSampleCondition(sampleObj, sample1_name, sample2_name, sample3_name, sample4_name):
    sampleLst = [sample1_name, sample2_name, sample3_name, sample4_name]
    
    plasticitySamples = []
    controlSamples = []
    
    for sampleName in sampleLst:
        for lst in sampleNameConditionLst:
            condition = lst[0]
            sample = lst[1]
            
            if sample == sampleName:
                if condition == 'Control': controlSamples.append(sampleName)
                else: plasticitySamples.append(sampleName)
    
    sampleObj.obs['condition'] = ['control' if sampleName in controlSamples else 'plasticity' for sampleName in sampleObj.obs['sample']]

# Plotting Functions Figure 6

In [ ]:
def annotatedClusteredUMAP(data, color, annotations, legend, ax):
    if legend == False:
        sc.pl.umap(data, color = color, show = False, ax=ax, legend_loc='right margin')
    else:
        sc.pl.umap(data, color = color, show = False, ax=ax)
    
    # Adding the cluster numbers to the graph (when the color is not set to 'leiden_0.3' the cluster nums are not disp.)
    if annotations == True:
        for i, cluster in enumerate(data.obs['leiden_0.3'].cat.categories):
            x = st.median(data.obsm['X_umap'][data.obs['leiden_0.3'] == cluster, 0])
            y = st.median(data.obsm['X_umap'][data.obs['leiden_0.3'] == cluster, 1])
            ax.text(x, y, cluster, fontsize=18, ha='center', va='center', fontweight='heavy')
    
    axisLabel_fontSize = 15
    plotTitle_fontSize = 20
    
    ax.set_title(f'Clustered UMAP Divided by {color}', fontsize=plotTitle_fontSize)
    
    if legend == True:
        patchA = mpatches.Patch(color='tab:blue', label='Female Samples')
        patchB = mpatches.Patch(color='tab:orange', label='Male Samples')
        ax.legend(handles=[patchA, patchB], loc='upper left')
    ax.set_xlabel('UMAP1', fontsize=axisLabel_fontSize)
    ax.set_ylabel('UMAP2', fontsize=axisLabel_fontSize)

In [ ]:
def figure6(sampleObj, title, condition, savePath):
    # Create a figure with subplots
    fig = plt.figure(figsize=(20,20), dpi=150)

    row = 6 
    col = 4

    ax1 = plt.subplot2grid((row, col), (0, 0), colspan=2, rowspan=2) # Clustered UMAP Seperated by Samples

    ax2 = plt.subplot2grid((row, col), (0, 2), colspan=2, rowspan=2) # Clustered UMAP Seperated by Sex

    ax3 = plt.subplot2grid((row, col), (2, 0), colspan=2, rowspan=2) # DE Genes Male

    ax4 = plt.subplot2grid((row, col), (2, 2), colspan=2, rowspan=2) # DE Genes Female 
    
    ax5 = plt.subplot2grid((row, col), (4, 0), colspan=2, rowspan=2) # DE Genes Plasticity

    ax6 = plt.subplot2grid((row, col), (4, 2), colspan=2, rowspan=2) # DE Genes Control 
    
    # Clustered UMAP Seperated by Samples
    annotatedClusteredUMAP(sampleObj, 'sample', True, False, ax2)
    
    # Clustered UMAP Seperated by Samples
    annotatedClusteredUMAP(sampleObj, 'sex', True, True, ax1)
    
    # DE Genes Male
    deGraph(sampleObj, 'male', ax3)
    
    # DE Genes Female
    deGraph(sampleObj, 'female', ax4)
        
    # Adds missing field to 'data.uns' that is necessary for running the ranked_sum_genes function
    sampleObj.uns['log1p'] = {'base': None}
    
    # Calculating DE Scores 
    sc.tl.rank_genes_groups(sampleObj, 'condition', method='wilcoxon')
        
    # DE Genes Plasticity
    deGraph(sampleObj, 'Control', ax5)
    
    # DE Genes Control
    deGraph(sampleObj, 'Deafened', ax6)
    
    # Figure Title
    fig.suptitle(f'{title})', fontsize=25,  fontweight='bold', y=0.92)
    
    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    
    # Saving the figure onto the computer 
    plt.savefig(os.path.join(savePath, f'{condition}_MvsF.jpg'), bbox_inches = 'tight')
    

In [ ]:
def makingFigure6(comparLst, savePath):
    for lst in comparLst:
        condition = lst[0]
        female_sample1_name = lst[1]
        female_sample2_name = lst[2]
        male_sample1_name = lst[3]
        male_sample2_name = lst[4]
        
        # Getting Sample AnnData Object 
        sampleObj = sc.read(f'F:/SampleData/PlasitcityComparisons_FvsM/{condition}.h5ad')
        
        # Calling Figure6 function 
        title = f'{condition} - Female Samples {female_sample1_name} & {female_sample2_name} vs. Male Samples {male_sample1_name} & {male_sample2_name}'
        figure6(sampleObj, title, condition, savePath)